In [56]:
import os
import pymongo
import pprint
from dotenv import load_dotenv
from bson.objectid import ObjectId

In [28]:
load_dotenv(".env")

True

In [29]:
MONGODB_URI = os.getenv("MONGODB_URI")

In [30]:
client = pymongo.MongoClient(MONGODB_URI)

In [31]:
for db in client.list_databases():
    print(db)

{'name': 'sample_airbnb', 'sizeOnDisk': 55443456, 'empty': False}
{'name': 'sample_analytics', 'sizeOnDisk': 9945088, 'empty': False}
{'name': 'sample_geospatial', 'sizeOnDisk': 2072576, 'empty': False}
{'name': 'sample_guides', 'sizeOnDisk': 40960, 'empty': False}
{'name': 'sample_mflix', 'sizeOnDisk': 118710272, 'empty': False}
{'name': 'sample_restaurants', 'sizeOnDisk': 7983104, 'empty': False}
{'name': 'sample_supplies', 'sizeOnDisk': 1097728, 'empty': False}
{'name': 'sample_training', 'sizeOnDisk': 54747136, 'empty': False}
{'name': 'sample_weatherdata', 'sizeOnDisk': 2842624, 'empty': False}
{'name': 'admin', 'sizeOnDisk': 335872, 'empty': False}
{'name': 'local', 'sizeOnDisk': 20017610752, 'empty': False}


In [32]:
db = client.sample_mflix

In [33]:
db.list_collection_names()

['movies', 'embedded_movies', 'theaters', 'comments', 'sessions', 'users']

In [35]:
users_collection = db.users

In [36]:
users_collection.find_one()

{'_id': ObjectId('59b99db4cfa9a34dcd7885b6'),
 'name': 'Ned Stark',
 'email': 'sean_bean@gameofthron.es',
 'password': '$2b$12$UREFwsRUoyF0CRqGNK0LzO0HM/jLhgUCNNIJ9RJAqMUQ74crlJ1Vu'}

### Inserting documents

In [37]:
new_user = {
    "name": "Billy Bob",
    "email": "billy_bean@gameofthron.es",
    "password": "1234"
}

In [44]:
result = users_collection.insert_one(new_user)

print(result.inserted_id)

67c4328fbe4daa83f262eb92


In [45]:
new_users = [
    {
        "name": "John Doe",
        "email": "john_doe@gameofthron.es",
        "password": "2345"
    },
    {
        "name": "Jane Doe",
        "email": "jane_doe@gameofthron.es",
        "password": "3456"
    }
]

In [46]:
result = users_collection.insert_many(new_users)

print(result.inserted_ids)

[ObjectId('67c43447be4daa83f262eb93'), ObjectId('67c43447be4daa83f262eb94')]


### Querying a collection

In [48]:
document_to_find = {"name": "John Doe"}

In [50]:
result = users_collection.find_one(document_to_find)

pprint.pprint(result)

{'_id': ObjectId('67c43447be4daa83f262eb93'),
 'email': 'john_doe@gameofthron.es',
 'name': 'John Doe',
 'password': '2345'}


In [ ]:
documents_to_find = {"name": {"$regex": " Doe"}}

In [55]:
cursor = users_collection.find(documents_to_find)

for doc in cursor:
    pprint.pprint(doc)

{'_id': ObjectId('67c43447be4daa83f262eb93'),
 'email': 'john_doe@gameofthron.es',
 'name': 'John Doe',
 'password': '2345'}
{'_id': ObjectId('67c43447be4daa83f262eb94'),
 'email': 'jane_doe@gameofthron.es',
 'name': 'Jane Doe',
 'password': '3456'}


### Updating documents

In [57]:
document_to_update = {"_id": ObjectId("67c43447be4daa83f262eb93")}

change_email = {"$set": {"email": "john_doe@breakingb.ad"}}

In [58]:
result = users_collection.update_one(document_to_update, change_email)

print(result.modified_count)

1


In [59]:
pprint.pprint(users_collection.find_one(document_to_update))

{'_id': ObjectId('67c43447be4daa83f262eb93'),
 'email': 'john_doe@breakingb.ad',
 'name': 'John Doe',
 'password': '2345'}


In [60]:
documents_to_update = {"email": {"$regex": "gameofthron.es"}}

change_password = {"$set": {"password": "4321"}}

In [61]:
result = users_collection.update_many(documents_to_update, change_password)

print(result.matched_count)
print(result.modified_count)

85
85


In [62]:
pprint.pprint(users_collection.find_one(documents_to_update))

{'_id': ObjectId('59b99db7cfa9a34dcd7885bd'),
 'email': 'aidan_gillen@gameofthron.es',
 'name': 'Petyr Baelish',
 'password': '4321'}


### Deleting documents

In [63]:
document_to_delete = {"_id": ObjectId("67c4328fbe4daa83f262eb92")}

In [64]:
pprint.pprint(users_collection.find_one(document_to_delete))

{'_id': ObjectId('67c4328fbe4daa83f262eb92'),
 'email': 'billy_bean@gameofthron.es',
 'name': 'Billy Bob',
 'password': '4321'}


In [65]:
result = users_collection.delete_one(document_to_delete)

print(result.deleted_count)

1


In [66]:
pprint.pprint(users_collection.find_one(document_to_delete))

None


In [67]:
documents_to_delete = {"name": {"$regex": " Doe"}}

In [68]:
pprint.pprint(users_collection.find_one(documents_to_delete))

{'_id': ObjectId('67c43447be4daa83f262eb93'),
 'email': 'john_doe@breakingb.ad',
 'name': 'John Doe',
 'password': '2345'}


In [69]:
result = users_collection.delete_many(documents_to_delete)

print(result.deleted_count)

2


In [70]:
pprint.pprint(users_collection.find_one(documents_to_delete))

None


### Aggregation

In [73]:
movies_collection = db.movies

In [74]:
movies_collection.find_one()

{'_id': ObjectId('573a1390f29313caabcd42e8'),
 'plot': 'A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels.',
 'genres': ['Short', 'Western'],
 'runtime': 11,
 'cast': ['A.C. Abadie',
  "Gilbert M. 'Broncho Billy' Anderson",
  'George Barnes',
  'Justus D. Barnes'],
 'poster': 'https://m.media-amazon.com/images/M/MV5BMTU3NjE5NzYtYTYyNS00MDVmLWIwYjgtMmYwYWIxZDYyNzU2XkEyXkFqcGdeQXVyNzQzNzQxNzI@._V1_SY1000_SX677_AL_.jpg',
 'title': 'The Great Train Robbery',
 'fullplot': "Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.",
 'languages': ['English'],
 'released': datetime.datetime(1903, 12, 1, 0, 0),
 'directors': ['Edwin S. Porter'],
 'rated': 'TV-G',
 'awards': {'wins': 1, 'nominations': 0, 

In [82]:
select_by_rated = {"$match": {"rated": "TV-G"}}

separate_by_type_sum_awards_wins = {
    "$group": {
        "_id": "$type",
        "total_wins": {"$sum": "$awards.wins"}
        }
}

pipeline = [select_by_rated, separate_by_type_sum_awards_wins]

In [83]:
results = movies_collection.aggregate(pipeline)

In [84]:
for doc in results:
    pprint.pprint(doc)

{'_id': 'movie', 'total_wins': 82}
{'_id': 'series', 'total_wins': 14}


In [107]:
select_movies = {
    "$match": {
        "genres": {"$in": ["Comedy", "Drama"]},
        "runtime": {"$lt": 20}
    }
}

sort_by_year = {"$sort": {"year": -1}}

return_specified_fields = {
    "$project": {
        "plot": 1,
        "title": 1,
        "countries": 1,
        "days": {"$multiply": ["$year", 365]},
        "_id": 0
    }
}

pipeline = [
    select_movies,
    sort_by_year,
    return_specified_fields
]

In [108]:
results = movies_collection.aggregate(pipeline)

In [109]:
for doc in results:
    pprint.pprint(doc)

{'countries': ['UK'],
 'days': 735475,
 'plot': 'Glen is barely a man. In a desperate attempt to tap into his '
         'masculinity he attends a primal scream therapy session, surrounded '
         'by wailing men he cannot even make a sound. When another ...',
 'title': 'Manoman'}
{'countries': ['USA'], 'days': 735475, 'title': 'Hallway'}
{'countries': ['France', 'Germany', 'Palestine'],
 'days': 735475,
 'plot': 'The silent routine of 5 Nuns living in the West Bank wilderness is '
         'disturbed when an Israeli settler family breaks down right outside '
         'the convent just as the Sabbath comes into effect.',
 'title': 'Ave Maria'}
{'countries': ['Austria', 'Japan'],
 'days': 735475,
 'plot': 'The dark year 2204, in a world that has seen 73 years of continuous '
         'war. A Shaman is sent on a mission to convert the soul of a giant '
         'battle colossus.',
 'title': 'The Shaman'}
{'countries': ['USA'],
 'days': 735475,
 'plot': 'A little girl is taken on a min